In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
import plotly.express as px
import seaborn as sns


df = pd.read_csv('superstore.csv', encoding='windows-1254')

df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
min_date = df['Order Date'].min()
max_date = df['Order Date'].max()

In [15]:
# Defining functions

def tranform_date(df, column):
    
    df = df.copy()
    
    to_datetime_column = pd.DataFrame(pd.to_datetime(df[column]))
    to_datetime_column[column + ' year'] = to_datetime_column[column].apply(lambda x: x.year)
    to_datetime_column[column + ' month'] = to_datetime_column[column].apply(lambda x: x.month)
    to_datetime_column[column + ' day'] = to_datetime_column[column].apply(lambda x: x.day)
    
    to_datetime_column.drop(column, axis=1, inplace=True)
    df.drop(column, axis=1, inplace=True)
    
    df = pd.concat([df, to_datetime_column], axis=1)
    
    return df


def preprocess_data(df):

    df = df.copy()
    
    # delete unecessary columns
    df = df.drop(['Row ID', 'Order ID', 'Customer ID','Customer Name', 'Postal Code',
                  'Country', 'Product ID', 'Product Name'], axis=1)

    # dealing with date column
    df = tranform_date(df, 'Order Date')
    df.drop('Order Date day', axis=1, inplace=True)

    return df

def category_counts(df, column, groupby_list=['Order Date year', 'Order Date month']):
    '''
        take a categorical column and create n categorical columns
        grouped by a list of columns.
    '''
    df = df.copy()
    
    aux_df = df.groupby(groupby_list)[column].value_counts().unstack().reset_index()
    new_columns = df[column].value_counts().index
    
    return pd.DataFrame(aux_df[new_columns].values, columns=new_columns)

def groupdata_month_year(df, groupdataby=['Order Date year', 'Order Date month']):
    '''
        group dataset by list of columns
    '''
    df = df.copy()
    
    aux_df = df.groupby(groupdataby).sum().reset_index()
    
    return aux_df

def onehot_encoding(df, column):
    df = df.copy()
    
    new_columns = pd.get_dummies(df[column], prefix=f'{column}')
    
    df.drop(column, axis=1, inplace=True)
    df = pd.concat([df, new_columns], axis=1)
    
    return df


In [16]:
# APPLYING COLUMN REMOVAL AND GROUPING BY MONTH AND YEAR

X = df.copy()

# remove unecessary columns and create other date columns
# unecessary columns are: Country, Row ID, Order IR, Customer ID, Customer Name, Product ID, Product Name
# create date columns: Year, Month, Day
X = preprocess_data(df)

# Get dataset grouped by month and year, numerical columns
df_numerical = groupdata_month_year(X)

# Dealing with categorical columns grouped by month and year
categorical_columns = ['Category', 'Sub-Category', 'Ship Mode', 'State', 'Segment'] # choose categorical columns
df_categorical = pd.DataFrame()

for column in categorical_columns:
    aux = category_counts(X, column).fillna(0) # counts categorical column items for each .groupby([month, year])
    df_categorical = pd.concat([df_categorical, aux], axis=1)
    
df_categorical.fillna(0, inplace=True)

# joining this new dataset
df_ = pd.concat([df_numerical, df_categorical], axis=1).fillna(0)

df_.head()

,Order Date year,Order Date month,Sales,Quantity,Discount,Profit,Office Supplies,Furniture,Technology,Binders,...,South Dakota,Vermont,District of Columbia,Maine,North Dakota,West Virginia,Wyoming,Consumer,Corporate,Home Office
0,2014,1,14236.895,284,10.00,2450.1907,50,20,9,12.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,49,13,17
1,2014,2,4519.892,159,8.10,862.3084,31,7,8,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27,12,7
2,2014,3,55691.009,585,26.30,498.7299,102,33,22,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78,49,30
3,2014,4,28295.345,536,14.85,3488.8352,83,26,26,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48,60,27
4,2014,5,23648.287,466,18.95,2738.7096,71,24,27,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71,37,14


In [17]:
from sklearn.preprocessing import StandardScaler

X_processed = df_.copy()

condition = X_processed['Order Date year'] < 2017
target = 'Profit'
year = 'Order Date year'


# Spliting data
X_train = X_processed[condition].drop([target, year], axis=1)
X_test = X_processed[(-condition)].drop([target, year], axis=1)
y_train = X_processed[condition][target]
y_test = X_processed[(-condition)][target]

# Scaling data
scaler = StandardScaler()
scaler.fit(X_train)

# Applying scaling
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)


In [18]:
from sklearn.metrics import mean_squared_error

# Size 24 means we will take the first 24 months of the dataset as traninig, and the rest of it for validations.
# That is, 2014 and 2015 will be training dataset, and 2016 will be validation dataset.
size=24

_X_train, _X_val, _y_train, _y_val = X_train_scaled[:size], X_train_scaled[size:], y_train[:size], y_train[size:]

In [19]:
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

# Set random state to 0
np.random.seed(0)

models = [LinearRegression(), RandomForestRegressor(), DecisionTreeRegressor(),
          GradientBoostingRegressor(), ElasticNet(), SVR(), Lasso(), Ridge(), XGBRegressor()]

model_names = ['Linear Regression', 'Random Forest', 'Decision Tree',
               'Gradient Boosting', 'Elastic Net', 'SVC', 'Lasso', 'Ridge', 'XGBoost']

model_rmse = []
models_ = []

for model in models:
    
    # Fit each model
    model.fit(_X_train, _y_train)
    models_.append(model)

    # Calculate RMSE (error)
    mse = mean_squared_error(_y_val, model.predict(_X_val))
    rmse = np.sqrt(mse)
    
    # Save values
    model_rmse.append(rmse)
    
df_models = pd.DataFrame(model_rmse, index=model_names, columns=['RMSE'])
df_models.sort_values('RMSE')

c:\Users\Mihir\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.987e+05, tolerance: 3.124e+04



,RMSE
Random Forest,4775.818047
XGBoost,4923.935136
Elastic Net,4993.861171
Ridge,5251.752659
Lasso,5270.881849
Linear Regression,5320.565564
Decision Tree,5336.037295
Gradient Boosting,5819.279755
SVC,5859.810614


In [20]:
def train_random_forest(params):
    max_depth = params[0]
    min_samples_split = params[1]
    min_samples_leaf = params[2]
    max_leaf_nodes = params[3]
    
    model = RandomForestRegressor(max_depth=max_depth, min_samples_split=min_samples_split,
                                 min_samples_leaf=min_samples_leaf, max_leaf_nodes=max_leaf_nodes,
                                 random_state=0, n_jobs=-1)
    
    model.fit(_X_train, _y_train)
    mse = mean_squared_error(_y_val, model.predict(_X_val))
    rmse = np.sqrt(mse)
    
    return rmse

def train_xboost(params):
    max_depth = params[0]
    learning_rate = params[1]
    colsample_bytree = params[2]
    subsample = params[3]
    
    model = XGBRegressor(max_depth=max_depth, learning_rate=learning_rate, colsample_bytree=colsample_bytree, 
                         subsample=subsample, n_estimators=100, random_state=0, n_jobs=-1)
    
    model.fit(_X_train, _y_train)
    mse = mean_squared_error(_y_val, model.predict(_X_val))
    rmse = np.sqrt(mse)
    
    return rmse

def train_elastic_net(params):
    alpha = params[0]
    l1_ratio = params[1]
    
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=10000, random_state=0)
    model.fit(_X_train, _y_train)
    mse = mean_squared_error(_y_val, model.predict(_X_val))
    rmse = np.sqrt(mse)
    
    return rmse

In [22]:
from skopt import gp_minimize
import warnings
warnings.filterwarnings('ignore')

# Random Forest params to explore
space_rf = [(2, 64, 'log-uniform'), #max_depth
         (2, 32), # min_sample_split
         (1, 32), # min_samples
         (2, 100)] # max_leaf_nodes

# XGBRegressor params to explore
space_xboost = [(2, 64), # max depth
         (1e-5, 1e-1), #learning rate
         (0.5, 1), #colsample
         (0.5, 1) #subsample
        ]

# Elastic net params to explore
space_elastic_net = [(0.5, 5.5), # alpha
         (0.1, 0.8) # l1_ratio
        ]

# Train models

rf_best_params = gp_minimize(train_random_forest, space_rf, random_state=1, verbose=0, n_calls=30, n_random_starts=10)

xgboost_best_params = gp_minimize(train_xboost, space_xboost, random_state=1, verbose=0, n_calls=30, n_random_starts=10)

elastic_net_best_params = gp_minimize(train_elastic_net, space_elastic_net, random_state=1, verbose=0, n_calls=30, n_random_starts=10)

# Print best params found in each model
print(f'Random Forest best params: max_depth={rf_best_params.x[0]}, min_sample_split={rf_best_params.x[1]}, min_samples={rf_best_params.x[2]},\
 max_leaf_nodes={rf_best_params.x[3]}')

print(f'XGBoost best params: max_depth={xgboost_best_params.x[0]}, learning_rate={xgboost_best_params.x[1]:.2f}, colsample={xgboost_best_params.x[2]:.2f}, \
subsample={xgboost_best_params.x[3]:.2f}')

print(f'Elastic net best params: alpha={elastic_net_best_params.x[0]}, l1_ratio={elastic_net_best_params.x[1]:.2f}')

Random Forest best params: max_depth=30, min_sample_split=14, min_samples=2, max_leaf_nodes=63
XGBoost best params: max_depth=27, learning_rate=0.09, colsample=0.50, subsample=1.00
Elastic net best params: alpha=5.5, l1_ratio=0.10


In [23]:
# Set models with best parameters
random_forest_regressor = RandomForestRegressor(max_depth=30, min_samples_split=14, min_samples_leaf=2, 
                                                max_leaf_nodes=63, n_jobs=-1, random_state=0, n_estimators=100)

xgboost = XGBRegressor(max_depth= 2, learning_rate=0.0743, colsample=0.5869, subsample=0.5, n_estimators=100, random_state=0, n_jobs=-1)

elastic_net = ElasticNet(alpha=5.5, l1_ratio=0.1, max_iter=10000, random_state=0)

models = [random_forest_regressor, xgboost, elastic_net]
models_error = []

for model in models:
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    # MSE
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    models_error.append(rmse)
    
print(models_error)

[12:10:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "colsample" } are not used.

[3179.145223955754, 2682.7731519993013, 3390.5657906023807]


In [27]:
print(X_train_scaled)

    Order Date month     Sales  Quantity  Discount  Office Supplies  \
0          -1.593255 -1.269347 -1.231848 -1.356087        -1.153912   
1          -1.303572 -1.691650 -1.597188 -1.491921        -1.508802   
2          -1.013890  0.532255 -0.352107 -0.190782        -0.182634   
3          -0.724207 -0.658365 -0.495321 -1.009355        -0.537524   
4          -0.434524 -0.860327 -0.699912 -0.716242        -0.761665   
5          -0.144841 -0.384575 -0.539162 -0.410975        -0.518845   
6           0.144841 -0.412769 -0.454403 -0.315891        -0.556202   
7           0.434524 -0.675135 -0.281962 -0.637602        -0.182634   
8           0.724207  1.665971  0.860824  0.993826         1.012786   
9           1.013890 -0.521116 -0.387180 -0.251549        -0.276026   
10          1.303572  1.529131  1.500901  1.365580         1.554461   
11          1.593255  1.134378  1.091719  1.721606         0.788645   
12         -1.593255 -1.098237 -1.372139 -1.333210        -1.490124   
13    

In [28]:
print(y_pred)

[ 2880.53969126  2927.85534356  7252.30299735  5261.72875179
  6906.85424424  6590.9587498   5368.95266265  6417.80280837
 12950.22786395  9393.55722985 12435.63947519 12880.88968558]
